In [256]:
%load_ext autoreload
%autoreload 2

%matplotlib widget
import matplotlib.pyplot as plt
from scipy.constants import R
import pandas as pd
import numpy as np
import os
import glob 

import ipywidgets as widgets

import pygaps as pg
from bet_class import BET

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
layout=widgets.Layout(width='50%', height='80px')
output2 = widgets.Output()

main_folder = widgets.Text(description='Main folder', layout=layout)

select_folder = widgets.Dropdown(description='Folder Select', layout=widgets.Layout(width='25%', height='30px'))
file_folder = widgets.SelectMultiple(description='File Select', layout=widgets.Layout(width='25%', height='300px'))


def on_value_change(change):
    folder = os.listdir(change.new)
    select_folder.options=folder

def on_value_change_2(change):
    fold = os.path.join(main_folder.value, change.new)
    files = [_ for _ in os.listdir(fold) if _.endswith('.DAT')]
    file_folder.options = files
    
main_folder.observe(on_value_change, names='value')
select_folder.observe(on_value_change_2, names='value')

main = widgets.VBox([main_folder, select_folder, file_folder, output2])    
display(main)

In [315]:
file_select = os.path.join(main_folder.value, select_folder.value, file_folder.value[0])

In [500]:
bet = BET(file_select)

In [501]:
bet.BET_analysis()

{'area': 289.17917209380806,
 'c_const': 734.3600790746036,
 'n_monolayer': 0.002964156251078749,
 'p_monolayer': 0.035588355132443024,
 'bet_slope': 336.9047329544369,
 'bet_intercept': 0.45939878999080896,
 'corr_coef': 0.9999948257096923,
 'pressure_range': [0.01276208106001559, 0.10198946752486834],
 'minimum': 10,
 'maximum': 16,
 'limits': [10, 16]}

In [497]:
isotherm_out = widgets.Output()
bet_plot = widgets.Output()
roq_plot = widgets.Output()
bet_res = widgets.Output()

with isotherm_out:
    bet.isotherm.plot()
    
with bet_plot:
    bet.plot_bet()
    
with roq_plot:
    bet.plot_roq()
    
with bet_res:
    display(pd.DataFrame(bet.BET_analysis()))
    
BET_plots = widgets.HBox([bet_plot, roq_plot])
tot_BET = widgets.VBox([BET_plots, bet_res])
main_out = widgets.HBox([isotherm_out, tot_BET])

display(main_out)